<a href="https://colab.research.google.com/github/NitzanEz/Final-Project/blob/main/classificationCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/NitzanEz/Final-Project.git


Cloning into 'Final-Project'...
remote: Enumerating objects: 89069, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 89069 (delta 9), reused 18 (delta 7), pack-reused 89044 (from 1)
Receiving objects: 100% (89069/89069), 1.08 GiB | 23.95 MiB/s, done.
Resolving deltas: 100% (1350/1350), done.
Updating files: 100% (114786/114786), done.


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import MaxPooling2D, Conv2D, AveragePooling2D
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten, Activation
from tensorflow.keras.layers import BatchNormalization, concatenate
from tensorflow.keras import regularizers, initializers, Model
from tensorflow.keras.utils import get_file

#########################################################################################
# Implements the Inception Network v4 (http://arxiv.org/pdf/1602.07261v1.pdf) in Keras. #
#########################################################################################

WEIGHTS_PATH = 'https://github.com/kentsommer/keras-inceptionV4/releases/download/2.1/inception-v4_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/kentsommer/keras-inceptionV4/releases/download/2.1/inception-v4_weights_tf_dim_ordering_tf_kernels_notop.h5'

def preprocess_input(x):
    x = np.divide(x, 255.0)
    x = np.subtract(x, 0.5)
    x = np.multiply(x, 2.0)
    return x

def conv2d_bn(x, nb_filter, num_row, num_col, padding='same', strides=(1, 1), use_bias=False):
    """
    Utility function to apply conv + BN.
    """
    channel_axis = -1 if tf.keras.backend.image_data_format() == 'channels_last' else 1
    x = Conv2D(nb_filter, (num_row, num_col),
               strides=strides,
               padding=padding,
               use_bias=use_bias,
               kernel_regularizer=regularizers.l2(0.00004),
               kernel_initializer=initializers.VarianceScaling(scale=2.0, mode='fan_in', distribution='normal', seed=None))(x)
    x = BatchNormalization(axis=channel_axis, momentum=0.9997, scale=False)(x)
    x = Activation('relu')(x)
    return x

def block_inception_a(input):
    channel_axis = -1 if tf.keras.backend.image_data_format() == 'channels_last' else 1

    branch_0 = conv2d_bn(input, 96, 1, 1)

    branch_1 = conv2d_bn(input, 64, 1, 1)
    branch_1 = conv2d_bn(branch_1, 96, 3, 3)

    branch_2 = conv2d_bn(input, 64, 1, 1)
    branch_2 = conv2d_bn(branch_2, 96, 3, 3)
    branch_2 = conv2d_bn(branch_2, 96, 3, 3)

    branch_3 = AveragePooling2D((3,3), strides=(1,1), padding='same')(input)
    branch_3 = conv2d_bn(branch_3, 96, 1, 1)

    x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=channel_axis)
    return x

def block_reduction_a(input):
    channel_axis = -1 if tf.keras.backend.image_data_format() == 'channels_last' else 1

    branch_0 = conv2d_bn(input, 384, 3, 3, strides=(2,2), padding='valid')

    branch_1 = conv2d_bn(input, 192, 1, 1)
    branch_1 = conv2d_bn(branch_1, 224, 3, 3)
    branch_1 = conv2d_bn(branch_1, 256, 3, 3, strides=(2,2), padding='valid')

    branch_2 = MaxPooling2D((3,3), strides=(2,2), padding='valid')(input)

    x = concatenate([branch_0, branch_1, branch_2], axis=channel_axis)
    return x

def block_inception_b(input):
    channel_axis = -1 if tf.keras.backend.image_data_format() == 'channels_last' else 1

    branch_0 = conv2d_bn(input, 384, 1, 1)

    branch_1 = conv2d_bn(input, 192, 1, 1)
    branch_1 = conv2d_bn(branch_1, 224, 1, 7)
    branch_1 = conv2d_bn(branch_1, 256, 7, 1)

    branch_2 = conv2d_bn(input, 192, 1, 1)
    branch_2 = conv2d_bn(branch_2, 192, 7, 1)
    branch_2 = conv2d_bn(branch_2, 224, 1, 7)
    branch_2 = conv2d_bn(branch_2, 224, 7, 1)
    branch_2 = conv2d_bn(branch_2, 256, 1, 7)

    branch_3 = AveragePooling2D((3,3), strides=(1,1), padding='same')(input)
    branch_3 = conv2d_bn(branch_3, 128, 1, 1)

    x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=channel_axis)
    return x

def block_reduction_b(input):
    channel_axis = -1 if tf.keras.backend.image_data_format() == 'channels_last' else 1

    branch_0 = conv2d_bn(input, 192, 1, 1)
    branch_0 = conv2d_bn(branch_0, 192, 3, 3, strides=(2, 2), padding='valid')

    branch_1 = conv2d_bn(input, 256, 1, 1)
    branch_1 = conv2d_bn(branch_1, 256, 1, 7)
    branch_1 = conv2d_bn(branch_1, 320, 7, 1)
    branch_1 = conv2d_bn(branch_1, 320, 3, 3, strides=(2,2), padding='valid')

    branch_2 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(input)

    x = concatenate([branch_0, branch_1, branch_2], axis=channel_axis)
    return x

def block_inception_c(input):
    channel_axis = -1 if tf.keras.backend.image_data_format() == 'channels_last' else 1

    branch_0 = conv2d_bn(input, 256, 1, 1)

    branch_1 = conv2d_bn(input, 384, 1, 1)
    branch_10 = conv2d_bn(branch_1, 256, 1, 3)
    branch_11 = conv2d_bn(branch_1, 256, 3, 1)
    branch_1 = concatenate([branch_10, branch_11], axis=channel_axis)

    branch_2 = conv2d_bn(input, 384, 1, 1)
    branch_2 = conv2d_bn(branch_2, 448, 3, 1)
    branch_2 = conv2d_bn(branch_2, 512, 1, 3)
    branch_20 = conv2d_bn(branch_2, 256, 1, 3)
    branch_21 = conv2d_bn(branch_2, 256, 3, 1)
    branch_2 = concatenate([branch_20, branch_21], axis=channel_axis)

    branch_3 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(input)
    branch_3 = conv2d_bn(branch_3, 256, 1, 1)

    x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=channel_axis)
    return x

def inception_v4_base(input):
    if tf.keras.backend.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1

    net = conv2d_bn(input, 32, 3, 3, strides=(2,2), padding='valid')
    net = conv2d_bn(net, 32, 3, 3, padding='valid')
    net = conv2d_bn(net, 64, 3, 3)

    branch_0 = MaxPooling2D((3,3), strides=(2,2), padding='valid')(net)

    branch_1 = conv2d_bn(net, 96, 3, 3, strides=(2,2), padding='valid')

    net = concatenate([branch_0, branch_1], axis=channel_axis)

    branch_0 = conv2d_bn(net, 64, 1, 1)
    branch_0 = conv2d_bn(branch_0, 96, 3, 3, padding='valid')

    branch_1 = conv2d_bn(net, 64, 1, 1)
    branch_1 = conv2d_bn(branch_1, 64, 1, 7)
    branch_1 = conv2d_bn(branch_1, 64, 7, 1)
    branch_1 = conv2d_bn(branch_1, 96, 3, 3, padding='valid')

    net = concatenate([branch_0, branch_1], axis=channel_axis)

    branch_0 = conv2d_bn(net, 192, 3, 3, strides=(2,2), padding='valid')
    branch_1 = MaxPooling2D((3,3), strides=(2,2), padding='valid')(net)

    net = concatenate([branch_0, branch_1], axis=channel_axis)

    # 4 x Inception-A blocks
    for i in range(4):
        net = block_inception_a(net)

    # Reduction-A block
    net = block_reduction_a(net)

    # 7 x Inception-B blocks
    for i in range(7):
        net = block_inception_b(net)

    # Reduction-B block
    net = block_reduction_b(net)

    # 3 x Inception-C blocks
    for i in range(3):
        net = block_inception_c(net)

    return net

def inception_v4(num_classes, dropout_keep_prob, weights, include_top):
    inputs = Input((299, 299, 3))
    x = inception_v4_base(inputs)

    if include_top:
        x = AveragePooling2D((8,8), padding='valid')(x)
        x = Dropout(dropout_keep_prob)(x)
        x = Flatten()(x)
        x = Dense(units=num_classes, activation='softmax')(x)

    model = Model(inputs, x, name='inception_v4')

    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('inception-v4_weights_tf_dim_ordering_tf_kernels.h5', WEIGHTS_PATH, cache_subdir='models', md5_hash='9fe79d77f793fe874470d84ca6ba4a3b')
        else:
            weights_path = get_file('inception-v4_weights_tf_dim_ordering_tf_kernels_notop.h5', WEIGHTS_PATH_NO_TOP, cache_subdir='models', md5_hash='9296b46b5971573064d12e4669110969')
        model.load_weights(weights_path, by_name=True)
    return model

def create_model(num_classes=1001, dropout_prob=0.2, weights='imagenet', include_top=True):
    return inception_v4(num_classes, dropout_prob, weights, include_top)


In [3]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def preprocess_mri_image(image_path):
    img = load_img(image_path, color_mode='grayscale', target_size=(299, 299))
    img_array = img_to_array(img)
    img_array = np.repeat(img_array, 3, axis=2)  # Repeat the grayscale data across three channels
    img_array = preprocess_input(img_array)  # Normalize using the provided function
    return img_array


In [4]:
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Assuming you have a base model setup already (e.g., InceptionV4)
inputs = Input(shape=(299, 299, 3))
base_model_output = inception_v4_base(inputs)  # Make sure this function is adjusted or correctly implemented

x = GlobalAveragePooling2D()(base_model_output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)  # Change this to 2 classes

model = Model(inputs, predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [5]:
# Define the paths to the training and validation directories
train_dir = '/content/Final-Project/Data/train'
validation_dir = '/content/Final-Project/Data/validation'

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up the image data generators with preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input  # Make sure this function is defined or imported
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

# Setup for validation data generator
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)


Found 76760 images belonging to 2 classes.
Found 21972 images belonging to 2 classes.


In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 11/100 ━━━━━━━━━━━━━━━━━━━━ 1:42:55 69s/step - accuracy: 0.4870 - loss: 4.3446

In [ ]:
# Evaluate the model
# Assume 'test_generator' is set up similarly to 'train_generator'
evaluation = model.evaluate(test_generator)

# Making predictions
image_path = '/content/Final-Project/Data/train/ASD/28764/slice_242.jpg'
processed_image = preprocess_mri_image(image_path)
processed_image = np.expand_dims(processed_image, axis=0)  # Add batch dimension for prediction
prediction = model.predict(processed_image)


In [ ]:
!git add classificationCode.ipynb  # Change 'Notebook.ipynb' to your file name
!git commit -m "Add Jupyter notebook"
!git push
